In [45]:
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from particle import Particle, InvalidParticle, ParticleNotFound
from particle_utils import get_charge_from_pdgc, get_name_from_pdgc

# Initial FASERvSi studies

Detector setup:
132 layers of Tungsten and 'SCT' (sensitive vacuum)
Tungsten layers are 0.9 mm thick
Gap between Tungsten sheets is 7.08 mm
1000 events generated - mix of $\nu_e$, $\nu_\mu$ and $\nu_\tau$


In [46]:
data = uproot.open("/home/bewilson/FASERvSi_G4/GeantOutput/FASERvSi-1000Taus.root")
hits = [data[treename] for treename in data.keys() if "Hits" in treename]
truth = data["truth"]

In [47]:
output_dir = "eventPlots/FASERvSi-1000Taus/"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
pdgc = hits[-1].arrays("pdgc", library="np")["pdgc"]

pdgc = set(pdgc)

for p in pdgc:
    print(f"pdgc = {p:<13}    name = {get_name_from_pdgc(p):<13}    charge = {get_charge_from_pdgc(p)}")

In [49]:
def scatter_on_ax(ax, event_data, x_var, y_var, label, color,  alpha=1, marker_size=1):
    
    if len(event_data[x_var]) == 0:
        return None
    
    return ax.scatter(event_data[x_var], event_data[y_var], color=color, marker=",", label=label, alpha=alpha, s=marker_size)

In [50]:
def set_offsets_on_scatter(scatter, event_data, x_var, y_var):
    
    if len(event_data[x_var]) == 0:
        return None
    
    return scatter.set_offsets(np.c_[event_data[x_var], event_data[y_var]])

In [51]:
event_numbers = truth.arrays("fEvent", library="np")["fEvent"]

In [52]:
neutrino_pdgc_to_label_dict = {12: r"$\nu_e$",
                               -12: r"$\bar{\nu}_e$",
                               14: r"$\nu_\mu$",
                               -14: r"$\bar{\nu}_\mu$",
                               16: r"$\nu_\tau$",
                               -16: r"$\bar{\nu}_\tau$",
                                }

In [ ]:

for i in tqdm(range(0, 100)):
    
    fig, ax = plt.subplots(ncols=2, figsize=(18, 6))

    truth_info = truth.arrays(truth.keys(), library="ak", cut=f"fEvent == {event_numbers[i]}")
    
    neutrino_pdgc = truth_info['nu_pdgc'][0]
    neutrino_energy = truth_info['nu_E'][0]
    target_pdgc = truth_info['target_pdgc'][0]
    event_number = truth_info['fEvent'][0]
    vx = truth_info['vertex_x']
    vy = truth_info['vertex_y']
    vz = truth_info['vertex_z']
    is_cc = bool(truth_info['isCC'][0])
    is_cc_label = "CC" if is_cc else "NC"
    
    ax[0].scatter(vz, vy, color="black", marker='x', s=1.5, label="primary vertex", zorder=100)
    ax[1].scatter(vx, vy, color="black", marker='x', s=1.5, label="primary vertex", zorder=100)
    
    for s, station in enumerate(hits):
        
        kinematics = station.arrays(station.keys(), library="ak", cut=f"(fEvent == {event_numbers[i]}) & (z > 0)")
        
        # print(kinematics)
        
        marker_size = 1
        alpha = 1
        
        electrons = kinematics[np.where(np.abs(kinematics['pdgc']) == 11)]
        muons = kinematics[np.where(np.abs(kinematics['pdgc']) == 13)]
        taus = kinematics[np.where(np.abs(kinematics['pdgc']) == 15)]
        
        gluons = kinematics[np.where(np.abs(kinematics['pdgc']) == 21)]
        photons = kinematics[np.where(np.abs(kinematics['pdgc']) == 22)]
        charged_hadrons = kinematics[np.where(np.abs(kinematics['pdgc']) > 37)]
        neutral_hadrons = kinematics[np.where(np.abs(kinematics['pdgc']) > 37)]
        charged_hadrons = kinematics[np.where(get_charge_from_pdgc(kinematics['pdgc']) != 0)]
        neutral_hadrons = kinematics[np.where(get_charge_from_pdgc(kinematics['pdgc']) == 0)]
        
        # scatter_on_ax(ax[0], neutral_hadrons, "z", "y", "neutral hadrons", 'grey', alpha=0.5, marker_size=marker_size)
        scatter_on_ax(ax[0], charged_hadrons, "z", "y", "charged hadrons", 'forestgreen', alpha=alpha, marker_size=marker_size)
        # scatter_on_ax(ax[0], photons, "z", "y", r"$\gamma$", 'yellow', alpha=0.5, marker_size=marker_size)
        # scatter_on_ax(ax[0], gluons, "z", "y", r"$g$", 'orange', alpha=0.5, marker_size=marker_size)
        scatter_on_ax(ax[0], electrons, "z", "y", r"$e^\pm$", 'lightblue', alpha=alpha, marker_size=marker_size)
        scatter_on_ax(ax[0], muons, "z", "y", r"$\mu^\pm$", 'tomato', alpha=alpha, marker_size=marker_size)
        scatter_on_ax(ax[0], taus, "z", "y", r"$\tau^\pm$", 'purple', alpha=alpha, marker_size=marker_size)
        
        # scatter_on_ax(ax[0], neutral_hadrons, "x", "y", "neutral hadrons", 'grey', alpha=0.5, marker_size=marker_size)
        scatter_on_ax(ax[1], charged_hadrons, "x", "y", "charged hadrons", 'forestgreen', alpha=alpha, marker_size=marker_size)
        # scatter_on_ax(ax[0], photons, "x", "y", r"$\gamma$", 'yellow', alpha=0.5, marker_size=marker_size)
        # scatter_on_ax(ax[0], gluons, "x", "y", r"$g$", 'orange', alpha=0.5, marker_size=marker_size)
        scatter_on_ax(ax[1], electrons, "x", "y", r"$e^\pm$", 'lightblue', alpha=alpha, marker_size=marker_size)
        scatter_on_ax(ax[1], muons, "x", "y", r"$\mu^\pm$", 'tomato', alpha=alpha, marker_size=marker_size)
        scatter_on_ax(ax[1], taus, "x", "y", r"$\tau^\pm$", 'purple', alpha=alpha, marker_size=marker_size)
        
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    
    ax[0].set_title(f"{is_cc_label} {neutrino_pdgc_to_label_dict[neutrino_pdgc]} + {get_name_from_pdgc(target_pdgc)}", loc="left")
    ax[0].set_title(f"Neutrino energy = {neutrino_energy:.2f} GeV", loc="right")
    ax[1].set_title(f"Event Number: {event_number}", loc="right")
    
    ax[0].legend(by_label.values(), by_label.keys())# , ncols=2) #, loc='center right', bbox_to_anchor=(1.55, 0.75))
    ax[1].legend(by_label.values(), by_label.keys())# , ncols=2) #, loc='center right', bbox_to_anchor=(1.55, 0.75)) 
    
    ax[0].set_xlabel("z position (mm)")
    ax[0].set_ylabel("y position (mm)")
    ax[0].set_xlim((550, 1600))
    ax[0].set_ylim((-150, 150))
    
    ax[1].set_xlabel("x position (mm)")
    ax[1].set_ylabel("y position (mm)")
    ax[1].set_xlim((-125, 125))
    ax[1].set_ylim((-150, 150))
    
    plt.savefig(f"{output_dir}/event_{i}_particle-dist-x-y-z-y.png", dpi=300, bbox_inches='tight')
    plt.show()
    plt.close()
    # break
        

In [ ]:
true_tau_events = truth.arrays(truth.keys(), library='ak', cut="(nu_pdgc == 16) | (nu_pdgc == -16)")    
print(true_tau_events['fEvent'])

In [11]:
# for event_number in tqdm(true_tau_events['fEvent']):
    
#     fig, ax = plt.subplots(ncols=2, figsize=(18, 6))

#     truth_info = truth.arrays(truth.keys(), library="ak", cut=f"fEvent == {event_number}")
    
#     neutrino_pdgc = truth_info['nu_pdgc'][0]
#     neutrino_energy = truth_info['nu_E'][0]
#     target_pdgc = truth_info['target_pdgc'][0]
#     event_number = truth_info['fEvent'][0]
#     vx = truth_info['vertex_x']
#     vy = truth_info['vertex_y']
#     vz = truth_info['vertex_z']
#     is_cc = bool(truth_info['isCC'][0])
#     is_cc_label = "CC" if is_cc else "NC"
    
#     ax[0].scatter(vz, vy, color="black", marker='x', s=1.5, label="primary vertex", zorder=100)
#     ax[1].scatter(vx, vy, color="black", marker='x', s=1.5, label="primary vertex", zorder=100)
    
#     for s, station in enumerate(hits):
        
#         kinematics = station.arrays(station.keys(), library="ak", cut=f"(fEvent == {event_number}) & (z > 0)")
        
#         # print(kinematics)
        
#         marker_size = 1
#         alpha = 1
        
#         electrons = kinematics[np.where(np.abs(kinematics['pdgc']) == 11)]
#         muons = kinematics[np.where(np.abs(kinematics['pdgc']) == 13)]
#         taus = kinematics[np.where(np.abs(kinematics['pdgc']) == 15)]
        
#         gluons = kinematics[np.where(np.abs(kinematics['pdgc']) == 21)]
#         photons = kinematics[np.where(np.abs(kinematics['pdgc']) == 22)]
#         charged_hadrons = kinematics[np.where(np.abs(kinematics['pdgc']) > 37)]
#         neutral_hadrons = kinematics[np.where(np.abs(kinematics['pdgc']) > 37)]
#         charged_hadrons = kinematics[np.where(get_charge_from_pdgc(kinematics['pdgc']) != 0)]
#         neutral_hadrons = kinematics[np.where(get_charge_from_pdgc(kinematics['pdgc']) == 0)]
        
#         # scatter_on_ax(ax[0], neutral_hadrons, "z", "y", "neutral hadrons", 'grey', alpha=0.5, marker_size=marker_size)
#         scatter_on_ax(ax[0], charged_hadrons, "z", "y", "charged hadrons", 'forestgreen', alpha=alpha, marker_size=marker_size)
#         # scatter_on_ax(ax[0], photons, "z", "y", r"$\gamma$", 'yellow', alpha=0.5, marker_size=marker_size)
#         # scatter_on_ax(ax[0], gluons, "z", "y", r"$g$", 'orange', alpha=0.5, marker_size=marker_size)
#         scatter_on_ax(ax[0], electrons, "z", "y", r"$e^\pm$", 'lightblue', alpha=alpha, marker_size=marker_size)
#         scatter_on_ax(ax[0], muons, "z", "y", r"$\mu^\pm$", 'tomato', alpha=alpha, marker_size=marker_size)
#         scatter_on_ax(ax[0], taus, "z", "y", r"$\tau^\pm$", 'purple', alpha=alpha, marker_size=marker_size)
        
#         # scatter_on_ax(ax[0], neutral_hadrons, "x", "y", "neutral hadrons", 'grey', alpha=0.5, marker_size=marker_size)
#         scatter_on_ax(ax[1], charged_hadrons, "x", "y", "charged hadrons", 'forestgreen', alpha=alpha, marker_size=marker_size)
#         # scatter_on_ax(ax[0], photons, "x", "y", r"$\gamma$", 'yellow', alpha=0.5, marker_size=marker_size)
#         # scatter_on_ax(ax[0], gluons, "x", "y", r"$g$", 'orange', alpha=0.5, marker_size=marker_size)
#         scatter_on_ax(ax[1], electrons, "x", "y", r"$e^\pm$", 'lightblue', alpha=alpha, marker_size=marker_size)
#         scatter_on_ax(ax[1], muons, "x", "y", r"$\mu^\pm$", 'tomato', alpha=alpha, marker_size=marker_size)
#         scatter_on_ax(ax[1], taus, "x", "y", r"$\tau^\pm$", 'purple', alpha=alpha, marker_size=marker_size)
        
#     handles, labels = plt.gca().get_legend_handles_labels()
#     by_label = dict(zip(labels, handles))
    
#     ax[0].set_title(f"{is_cc_label} {neutrino_pdgc_to_label_dict[neutrino_pdgc]} + {get_name_from_pdgc(target_pdgc)}", loc="left")
#     ax[0].set_title(f"Neutrino energy = {neutrino_energy/1e3:.2f} GeV", loc="right")
#     ax[1].set_title(f"Event Number: {event_number}", loc="right")
    
#     ax[0].legend(by_label.values(), by_label.keys())# , ncols=2) #, loc='center right', bbox_to_anchor=(1.55, 0.75))
#     ax[1].legend(by_label.values(), by_label.keys())# , ncols=2) #, loc='center right', bbox_to_anchor=(1.55, 0.75)) 
    
#     ax[0].set_xlabel("z position (mm)")
#     ax[0].set_ylabel("y position (mm)")
#     ax[0].set_xlim((550, 1600))
#     ax[0].set_ylim((-150, 150))
    
#     ax[1].set_xlabel("x position (mm)")
#     ax[1].set_ylabel("y position (mm)")
#     ax[1].set_xlim((-125, 125))
#     ax[1].set_ylim((-150, 150))
    
#     plt.savefig(f"{output_dir}/event_{i}_particle-dist-x-y-z-y.png", dpi=300, bbox_inches='tight')
#     plt.show()
#     plt.close()
#     # break
        

In [ ]:
# %matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
import matplotlib as mpl
# Set the ffmpeg path
plt.rcParams["animation.html"] = "jshtml"


# Create a figure and axis
fig, ax = plt.subplots()
x = np.linspace(0, 2 * np.pi, 100)
line, = ax.plot(x, np.sin(x))

# Update function for animation
def update(frame):
    line.set_ydata(np.sin(x + frame / 10.0))  # Update the data.
    return line,

# Create the animation
ani = animation.FuncAnimation(fig, update, frames=100, blit=True)
from IPython.display import HTML
HTML(ani.to_jshtml())
# plt.show()

# Save the animation (optional)
# ani.save('animation.mp4', writer='ffmpeg') // NOTE: Saving animation blocks inline animation from being rendered




In [13]:
def get_station_charged_hits(station, event_number):
    kinematics = station.arrays(station.keys(), library="ak", cut=f"(fEvent == {event_number}) & (z > 0)")
    electrons = kinematics[np.where(np.abs(kinematics['pdgc']) == 11)]
    muons = kinematics[np.where(np.abs(kinematics['pdgc']) == 13)]
    taus = kinematics[np.where(np.abs(kinematics['pdgc']) == 15)]
    
    charged_hadrons = kinematics[np.where(np.abs(kinematics['pdgc']) > 37)]
    charged_hadrons = kinematics[np.where(get_charge_from_pdgc(kinematics['pdgc']) != 0)]
    
    return electrons, muons, taus, charged_hadrons
    

In [ ]:
# %matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"

data = uproot.open("/home/bewilson/FASERvSi_G4/GeantOutput/FASERvSi.1000.HITS.root")
hits = [data[treename] for treename in data.keys() if "Hits" in treename]
truth = data["truth"]

def make_hit_animation(hits, truth, event_number):
    
    # Create a figure and axis
    fig, ax = plt.subplots()
    marker_size = 1
    alpha = 1
    os.makedirs(f"{event_number}_frames", exist_ok=True)
    
    # Get truth data
    truth_info = truth.arrays(truth.keys(), library='ak', cut=f"fEvent == {event_number}")
    neutrino_pdgc = truth_info['nu_pdgc'][0]
    neutrino_energy = truth_info['nu_E'][0]
    target_pdgc = truth_info['target_pdgc'][0]
    event_number = truth_info['fEvent'][0]
    vx = truth_info['vertex_x']
    vy = truth_info['vertex_y']
    vz = truth_info['vertex_z']
    
    # Setup plot
    charged_had_scatter  = ax.scatter([], [], label="charged hadrons", color='forestgreen', alpha=alpha, s=marker_size, marker=',')
    electron_scatter     = ax.scatter([], [], label=r"$e^\pm$", color='lightblue', alpha=alpha, s=marker_size, marker=',')
    muon_scatter         = ax.scatter([], [], label=r"$\mu^\pm$", color='tomato', alpha=alpha, s=marker_size, marker=',')
    tau_scatter          = ax.scatter([], [], label=r"$\tau^\pm$", color='purple', alpha=alpha, s=marker_size, marker=',')
    vtx_scatter          = ax.scatter([], [], label="primary vertex", color='black', alpha=alpha, s=marker_size, marker='x')
    ax.set_xlabel("x position (mm)")
    ax.set_ylabel("y position (mm)")
    ax.set_xlim((-125, 125))
    ax.set_ylim((-150, 150))
    ax.set_title(f"{neutrino_pdgc_to_label_dict[neutrino_pdgc]} + {get_name_from_pdgc(target_pdgc)}    Neutrino energy = {neutrino_energy/1e3:.2f} GeV    Event Number: {event_number}    SCT Layer = {1}", loc="left", fontsize=7)
    
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())# , ncols=2) #, loc='center right', bbox_to_anchor=(1.55, 0.75))
    
    # Init function
    def init():
        charged_had_scatter.set_offsets(np.empty((0, 2)))
        electron_scatter.set_offsets(np.empty((0, 2)))   
        muon_scatter.set_offsets(np.empty((0, 2)))       
        tau_scatter.set_offsets(np.empty((0, 2)))        
        vtx_scatter.set_offsets(np.empty((0, 2)))        
        
        return charged_had_scatter, electron_scatter, muon_scatter, tau_scatter, vtx_scatter     
    
    # Update function for animation
    def update(frame, hits, truth_info, event_number):
        kinematics = hits[frame].arrays(hits[frame].keys(), library="ak", cut=f"(fEvent == {event_number}) & (z > 0)")
        
        # print(frame, kinematics) #hits[frame].arrays(hits[frame].keys() , library="ak"))
        
        neutrino_pdgc = truth_info['nu_pdgc'][0]
        neutrino_energy = truth_info['nu_E'][0]
        target_pdgc = truth_info['target_pdgc'][0]
        event_number = truth_info['fEvent'][0]
        vx = truth_info['vertex_x']
        vy = truth_info['vertex_y']
        vz = truth_info['vertex_z']
        
        electrons = kinematics[np.where(np.abs(kinematics['pdgc']) == 11)]
        muons = kinematics[np.where(np.abs(kinematics['pdgc']) == 13)]
        taus = kinematics[np.where(np.abs(kinematics['pdgc']) == 15)]
        
        charged_hadrons = kinematics[np.where(np.abs(kinematics['pdgc']) > 37)]
        charged_hadrons = kinematics[np.where(get_charge_from_pdgc(kinematics['pdgc']) != 0)]
        
        charged_had_scatter.set_offsets(np.c_[charged_hadrons["x"], charged_hadrons["y"]])
        electron_scatter.set_offsets(np.c_[electrons["x"], electrons['y']])
        muon_scatter.set_offsets(np.c_[muons["x"], muons['y']])
        tau_scatter.set_offsets(np.c_[taus["x"], taus['y']])
        
        if len(electrons) != 0 or len(muons) != 0 or len(taus) != 0 or len(charged_hadrons) != 0:
            vtx_scatter.set_offsets(np.c_[vx, vy])

        fig.savefig(f"{event_number}_frames/{event_number}-hit-layer-{frame}.png", dpi=300)
        ax.set_title(f"{neutrino_pdgc_to_label_dict[neutrino_pdgc]} + {get_name_from_pdgc(target_pdgc)}    Neutrino energy = {neutrino_energy/1e3:.2f} GeV    Event Number: {event_number}    SCT Layer = {frame+1}", loc="left", fontsize=7)
        return charged_had_scatter, electron_scatter, muon_scatter, tau_scatter, vtx_scatter

    # Create the animation
    ani = animation.FuncAnimation(fig, update, frames=tqdm(range(len(hits))), blit=True, init_func=init, fargs=(hits, truth_info, event_number))
    from IPython.display import HTML
    HTML(ani.to_jshtml())
    # plt.show()

    # Save the animation (optional)
    ani.save(f'{event_number}.mp4', writer='ffmpeg') # NOTE: Saving animation blocks inline animation from being rendered

make_hit_animation(hits, truth, 92600902)


In [42]:
import math

def compute_tau_decay_length(energy_in_gev):
    # Assuming all momentum in z-direction
    energy_in_joules = energy_in_gev * 1e9 * 1.60217663 * 1e-19    # Joules
    speed_of_light = 299792458                                     # m / s
    tau_mass_in_kg = 1776.86 * 1e6 * 1.60217663 * 1e-19 / speed_of_light**2  # kg
    tau_lifetime_at_rest = 	2.903 * 1e-13                                  # s
    vz_squared = energy_in_joules**2 / (tau_mass_in_kg**2 * speed_of_light**2) - speed_of_light**2
    gamma = 1 / math.sqrt(1 - vz_squared / speed_of_light**2)
    tau_lifetime_lab_frame = gamma * tau_lifetime_at_rest
    
    decay_length = tau_lifetime_lab_frame * math.sqrt(vz_squared)
    
    return decay_length, gamma
    

def print_decay_length(energy_in_gev):
    decay_length, gamma = compute_tau_decay_length(energy_in_gev)
    print(f"Tau with energy {energy_in_gev:<8} GeV has a gamma factor of {gamma:<8.2f} and an averge decay length of {decay_length*1000:<8.4f} mm")

In [ ]:
print_decay_length(1.88)